## This notebook is scarchpad for new_kernel3

The idea is that computational complexity for searching connected states from a given state reduce to $O(1)$ from $O(L^2)$ by using the information from the one before. 

#### We are going the test the new kenel called "kernel3" for 12 x 12 system

In [1]:
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics, new_dynamics2
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)
from conf import *

In [2]:
length = [4, 4]
hex_ = nk.machine.new_hex(np.array(length))
kernel = 1
decay_factor = 1  # or 'sigmoid decay'
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
op, ad2o_o, op_num, label_num  = f.dimer_flip1(length = np.array(length), return_info = True)

ad2_bool = np.zeros([ad2o_o.shape[0], ad2o_o.shape[0]], dtype = np.bool)
for l in range(ad2o_o.shape[0]):
    label = ad2o_o[l]
    for op_ in label:
        ad2_bool[l,op_] = True


ma = nk.machine.RbmDimer(hi, hex_, alpha = 2, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.hex.ad2o_o = ad2o_o.astype(np.int64)
ma.hex.ad2_bool = ad2_bool
ma.init_random_parameters(seed=1234)

2


In [3]:
V = 1.0
h = 1.0

ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))


n_jobs = 12

sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op
    , length = length, n_chains=n_chains, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sampler = sa_mul.sa_list[0]



yoyo
number of core : 12


## measure execution time for transition function

In [4]:
# state = np.ones(kernel.size).reshape(1,-1)
# state1 = state.copy()

# %timeit kernel.transition(state, state1, sampler._w, None, 10**4)
# accepted

In [5]:
# %timeit kernel.new_transition(state, state1, sampler._w, None, 10**4)

In [6]:
import netket.sampler.numpy.local_kernel as local_kernel
sweep_size = np.random.randint(0, 10) + sampler.sweep_size
importlib.reload(local_kernel)

kernel = local_kernel._DimerLocalKernel_1
kernel = kernel(np.array(ma.hilbert.local_states, dtype=np.int8),ma.input_size,
                op._basis, op._constant, op._diag_mels, op._n_conns, op._mels, 
                op._x_prime, op._acting_on, op._acting_size, ad2o_o.astype(np.int64), ad2_bool)
sampler.reset()
state = sampler._state
state1 = sampler._state1
sections_1 = np.zeros(1, dtype=np.int64)

In [7]:
op._x_prime.shape

(96, 16, 16, 4)

In [8]:
importlib.reload(local_kernel)

state = np.ones(kernel.size)
state1 = state.copy()
%timeit kernel.new_transition_2(state, state1, ma._w, None, 10**4)

23.9 ms ± 64.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
state

array([ 1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1., -1.,  1., -1.,
       -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,
       -1., -1., -1., -1., -1., -1.])

### measure components of transition functions
- get_conn
- _log_val_kern

each functions are executed sweep_size times

### As seen from above results, almost all calculation time comes from these two functions


### First, optimize get_conn

Since it is applied consectively, we don't need to care about local operators away from the point where last local operator is applied. In the following, I restrict the dimer model to 4 x 4

In [10]:
import netket.machine.functions2 as f2
importlib.reload(f2)
importlib.reload(f)

<module 'scripts.functions' from '../scripts/functions.py'>

In [11]:
length = [14, 14]
hex_ = f2.new_hex(np.array(length))
kernel = 1
decay_factor = 1  # or 'sigmoid decay'
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
op = f.dimer_flip1(length = np.array(length))

ma = nk.machine.RbmDimer(hi, hex_, alpha = 2, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)

2


#### create function to fetch related operator by given one local operator


We first need to update new_hex function in order to deal with it

In [12]:
op, ad2o_o, op_num, label_num  = f.dimer_flip1(length = np.array(length), return_info = True)
ad2_bool = np.zeros([ad2o_o.shape[0], ad2o_o.shape[0]], dtype = np.bool)
for l in range(ad2o_o.shape[0]):
    label = ad2o_o[l]
    for op_ in label:
        ad2_bool[l,op_] = True
np.random.seed(2021)


then, modify get_conn_flattered_kernel function

In [13]:
importlib.reload(local_kernel)
kernel = local_kernel._DimerLocalKernel_1
kernel = kernel(np.array(ma.hilbert.local_states, dtype=np.int8),ma.input_size,
                op._basis, op._constant, op._diag_mels, op._n_conns, op._mels, 
                op._x_prime, op._acting_on, op._acting_size, ad2o_o.astype(np.int64), ad2_bool)

In [14]:
state = np.ones(kernel.size).reshape(1,-1)
state_1 = state.copy()
# state = sampler._state.reshape(1,-1)
sections = np.zeros(1, dtype=np.int32)
op_labels = np.arange(len(op._acting_on))

In [15]:
x_prime1, acting_on, conn_op = local_kernel.new_get_conn_one(
                                        state,
                                        sections,
                                        kernel.basis,
                                        kernel.n_conns,
                                        kernel.x_prime,
                                        kernel.acting_on,
                                        op_labels)

In [16]:
x_prime2 = local_kernel.get_conn_one(
                                    state,
                                    sections,
                                    kernel.basis,
                                    kernel.n_conns,
                                    kernel.x_prime,
                                    kernel.acting_on,
                                )


In [17]:
states = np.zeros((int(kernel.size/4), kernel.size))
x_prime = np.zeros((int(kernel.size/4), kernel.size))
state = np.ones(kernel.size).reshape(1,-1)
# state = sampler._state.reshape(1,-1)
sections = np.zeros(1, dtype=np.int32)
op_labels = np.arange(len(op._acting_on))

In [18]:
# %%timeit
conn_op3 = local_kernel.get_conn_one_2(
                                        state,
                                        sections,
                                        kernel.basis,
                                        kernel.n_conns,
                                        kernel.x_prime,
                                        kernel.acting_on,
                                        op_labels,
                                        x_prime)

#### Note that # of connected states is bounded by $(L \times L / 2)$

In [19]:
importlib.reload(local_kernel)

def transition(state, state_1, w, r, sweep_size, _basis, _n_conns, _x_prime, _acting_on):
    '''
    This transition is exclusively for batch_size = 1
    '''

    accepted = 0
    batch_size = state.shape[0]

    assert batch_size == 1, 'batch_size must be 1'

    sections = np.zeros(1, dtype=np.int64)
    sections_1 = np.zeros(1, dtype=np.int64)
    state_1 = state
    r = np.zeros((1, w.shape[1]), dtype = np.float64)
    
    log_values = _log_val_kernel(state.astype(np.float64), w, r)[0]
#         print(log_values)

    state_prime = local_kernel.get_conn_one(state,sections, _basis, _n_conns, _x_prime, _acting_on)
    n_conn = sections[0]-1
#         print(log_values_prime)

    N = 0
    for _ in range(sweep_size * 2):
    # while True:

#             state_prime_, mels_ = self.get_conn(state, sections)
#             n_conn_ = sections[0]-1

        # print((state_prime_ == state_prime).any())
        # print((n_conn_ + 1).sum())
#             print(log_values)

        rs = (np.random.rand(1) * (n_conn)).astype(np.int64)

        state_1 = state_prime[rs+1].reshape(1,-1)

        state_1_prime = local_kernel.get_conn_one(state_1, sections_1, _basis, _n_conns, _x_prime, _acting_on)
        n_conn_1 = sections_1[0]-1

        prob_corr = n_conn/n_conn_1


        log_values_1 = _log_val_kernel(state_1.astype(np.float64), w, r)[0]

        prob = np.exp(
            2 * (log_values_1 - log_values)
        ) * prob_corr

        if prob > np.random.rand(1):
            state[:] = state_1
            state_prime = state_1_prime
            log_values = log_values_1
            n_conn = n_conn_1
            accepted += 1


        # sections = sections_1

        N += 1
        if accepted >= sweep_size:
            break

    return accepted/N

In [20]:
from numba import boolean

In [21]:
from numba import njit
importlib.reload(local_kernel)

# @njit
def new_transition(state, state_1, w, r, sweep_size, _basis, _n_conns, _x_prime, _acting_on, _ad2o_o, _ad2_bool):
    '''
    This transition is exclusively for batch_size = 1
    '''

    accepted = 0
    batch_size = state.shape[0]
    sys_size = state.shape[-1]

    assert batch_size == 1, 'batch_size must be 1'

    sections = np.zeros(1, dtype=np.int64)
    sections_1 = np.zeros(1, dtype=np.int64)
    state_1 = state
    
    r = np.zeros((1, w.shape[1]), dtype = np.float64)

    log_values = _log_val_kernel(state.astype(np.float64), w, r)[0]
    
    state_prime = np.zeros((int(sys_size/4), sys_size), dtype=np.int32)
    state_1_prime = np.zeros((int(sys_size/4), sys_size), dtype=np.int32)
#     state_prime[:] = state
    
    
    conn_op_prime = np.zeros(int(sys_size/4), dtype=np.int32)
    conn_op_prime_1 = np.zeros(int(sys_size/4), dtype=np.int32)
    

    op_labels = np.arange(len(_acting_on))
#     print(state_prime.shape)
    conn_op_prime[:] = local_kernel.get_conn_one_2(
                                    state, sections,_basis,_n_conns,_x_prime,_acting_on, op_labels, state_prime)
    

    
    n_conn = sections[0]
    n_conn_1 = n_conn
        
    np.random.seed(2021)
    last_state = state

    N = 0
    last_picked = 0
    for kk in range(sweep_size * 2):
#         print(kk)

#         print(f'conn_op_prime (this is labels of flippable operators in last op_labels) = {conn_op_prime}')
#         print(f"n_conn = {n_conn}")
#         print(state)
        rs = (np.random.rand(1)[0] * n_conn).astype(np.int64)
        op_label = conn_op_prime[rs] # choose operator label randomly. and flip the plaquete the choosen operator acting on.
        op_labels = _ad2o_o[op_label] # list up all adjacent operators to choosen operator.
        
        state_old = state
        state_1 = state_prime[rs].reshape(1,-1).copy() # new candidate state
        op_label_bool = _ad2_bool[op_label]
#         print((state_1==0).any())

#         print(conn_op_prime, n_conn, op_label, last_picked, list(last_state[0]))
    

        local_kernel.get_conn_one_3(
                                    state_1, sections,_basis,_n_conns,
                                    _x_prime,_acting_on, op_labels, state_1_prime,
                                    conn_op_prime, conn_op_prime_1, op_label_bool, n_conn)
        n_conn_1 = sections[0]
        
        
        
        prob_corr = n_conn/n_conn_1


        log_values_1 = _log_val_kernel(state_1.astype(np.float64), w, r)[0]

        prob = np.exp(
            2 * (log_values_1 - log_values)
        ) * prob_corr

        if prob > np.random.rand(1):

            state[:] = state_1
            state_prime[:] = state_1_prime
            conn_op_prime[:] = conn_op_prime_1

            log_values = log_values_1
            n_conn = n_conn_1
            accepted += 1
        
#         else:
#             print("YO?")
            
#         print("-" * 20)
#         print("\n\n")

        N += 1
        if accepted >= sweep_size:
            break
        
    return accepted/N

@njit
def _log_val_kernel(x, W, r):

    if x.ndim != 2:
        raise RuntimeError("Invalid input shape, expected a 2d array")

    # if out is None:
    out = np.empty(x.shape[0], dtype=np.float64)
    r[:] = x.dot(W)
    _log_cosh_sum(r, out)

    return out

@njit(fastmath=True)
def _log_cosh_sum(x, out, add_factor=None):
    x = x * np.sign(x.real)
    if add_factor is None:
        for i in range(x.shape[0]):
            out[i] = np.sum(x[i] - np.log(2.0) + np.log(1.0 + np.exp(-2.0 * x[i])))
    else:
        for i in range(x.shape[0]):
            out[i] += add_factor * (
                np.sum(x[i] - np.log(2.0) + np.log(1.0 + np.exp(-2.0 * x[i])))
            )

    return out

In [22]:
state = np.ones((1,kernel.size),dtype=np.int32)
state1 = np.copy(state)

In [23]:
ad2_bool = np.zeros([ad2o_o.shape[0], ad2o_o.shape[0]], dtype = np.bool)
for l in range(ad2o_o.shape[0]):
    label = ad2o_o[l]
    for op_ in label:
        ad2_bool[l,op_] = True

In [24]:
state = np.ones((1,kernel.size),dtype=np.int32)
tmp = new_transition(state, state1, ma._w, None, 10**4, kernel.basis,kernel.n_conns, kernel.x_prime, kernel.acting_on, ad2o_o.astype(np.int64), ad2_bool)

In [ ]:
w_.shape

In [68]:
%timeit op._acting_on[acting_on]

182 ns ± 0.64 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [151]:
%load_ext line_profiler

In [155]:
state = np.ones((1,kernel.size),dtype=np.int32)
state1 = np.copy(state)
%lprun -T profile3.txt -f new_transition new_transition(state, state1, ma._w, None, 10**4, kernel.basis,kernel.n_conns, kernel.x_prime, kernel.acting_on, ad2o_o, ad2_bool)


*** Profile printout saved to text file 'profile3.txt'. 


In [175]:
state = np.ones(kernel.size,dtype=np.int32)
state1 = np.copy(state)
%lprun -T profile4.txt -f new_transition_2 new_transition_2(state, state1, ma._w, None, 10**4, kernel.basis,kernel.n_conns, kernel.x_prime, kernel.acting_on, ad2o_o, ad2_bool)


*** Profile printout saved to text file 'profile4.txt'. 


In [223]:
state = np.ones((1,kernel.size),dtype=np.int32)
state1 = np.copy(state)
%lprun -T profile2.txt -f transition transition(state, state1, ma._w, None, 10**4, kernel.basis,kernel.n_conns, kernel.x_prime, kernel.acting_on)


*** Profile printout saved to text file 'profile2.txt'. 


### Second, optimize _log_val_kernel function. 
It calculate x.dot(ma._w) $\leftarrow$ measure execution time.

In [24]:
from numba import njit
@njit
def _new_log_val_kernel_2(x, W, r_prime, r_prime_1, acting_on):
    
    '''
    r_prime : last r_prime
    r_prime_1 : new r_prime
    '''
    
#     W_prime = W[acting_on, :]
    
    assert x.ndim == 1, "Invalid input shape, expected a 1d array"
#     assert x_prime.shape[0] == 2, 'x should be 1 by 2 matrix'
    r_prime_1[:] = 2 * (x[acting_on[0]] * W[acting_on[0]] + x[acting_on[1]] * W[acting_on[1]]) + r_prime
    out = _new_log_cosh_sum(r_prime_1)

    return out

@njit
def _new_log_val_kernel_1(x, W, r):

    assert x.ndim == 1, "Invalid input shape, expected a 1d array"


    # if out is None:
    r[:] = x.dot(W)
    out = _new_log_cosh_sum(r)

    return out

@njit(fastmath=True)
def _new_log_cosh_sum(x, add_factor=None):
    x = np.abs(x)
    out = np.sum(x + np.log(1.0 + np.exp(-2.0 * x))) - x.shape[0] * np.log(2.0)
    return out

@njit
def w_acting_on(w, acting_on_list):
    
    shape = (acting_on_list.shape[0], 2, w.shape[1])
    w_ = np.empty(shape, dtype = np.float64)
    for i in range(shape[0]):
        w_[i] = w[acting_on_list[i], :]
    
    return w_

In [25]:
importlib.reload(local_kernel)

def new_transition_2(state, state_1, w , r, sweep_size, _basis, _n_conns, _x_prime, _acting_on, _ad2o_o, _ad2_bool):
    '''
    This transition is exclusively for batch_size = 1
    w_ :  w_acting_on(w, _acting_on)
    '''

    accepted = 0
    sys_size = state.shape[0]
#     _w =  w_acting_on(w, _acting_on[:,1:3])


    sections = np.zeros(1, dtype=np.int64)
    sections_1 = np.zeros(1, dtype=np.int64)
    state_1[:] = state
    
    r_prime = np.zeros(w.shape[1], dtype = np.float64)
    r_prime_1 = np.zeros(w.shape[1], dtype = np.float64)
    
    log_values = _new_log_val_kernel_1(state.astype(np.float64), w, r_prime)
    
#     w_ = w_acting_on(w, _acting_on)

    state_prime = np.zeros((int(sys_size/4), sys_size), dtype=np.int32)
    
    
    conn_op_prime = np.zeros(int(sys_size/4), dtype=np.int32)
    conn_op_prime_1 = np.zeros(int(sys_size/4), dtype=np.int32)
    
    acting_on_prime = np.zeros((int(sys_size/4), 4), dtype=np.int32)
    acting_on_prime_1 = np.zeros((int(sys_size/4), 4), dtype=np.int32)
    
    
    
    op_labels = np.arange(len(_acting_on))
    conn_op_prime[:] = local_kernel.get_conn_one_4(
                                    state, sections,_basis,_n_conns,_acting_on, op_labels, acting_on_prime)

    
    n_conn = sections[0]
    n_conn_1 = n_conn
        
    np.random.seed(2021)
    last_state = state

    N = 0
    last_picked = 0
    for kk in range(sweep_size * 2):

#         print(state)

        
        rs = (np.random.rand(1)[0] * n_conn).astype(np.int64)
        op_label = conn_op_prime[rs] # choose operator label randomly. and flip the plaquete the choosen operator acting on.
#         w_prime = _w[op_label]
        op_labels = _ad2o_o[op_label] # list up all adjacent operators to choosen operator.
        state_old = state
        state_1[:] = state
#         print(rs)
#         print(r_prime[:10])
        acting_on_rs = acting_on_prime[rs][1:3]
#         print(state_1)
        state_1[acting_on_rs] *= -1
        log_values_1 = _new_log_val_kernel_2(state_1.astype(np.float64), w, r_prime, r_prime_1, acting_on_rs)
        
        
        op_label_bool = _ad2_bool[op_label] 
        local_kernel.get_conn_one_5(
                                    state_1, sections,_basis,_n_conns,
                                    _acting_on, op_labels,conn_op_prime,
                                     conn_op_prime_1,acting_on_prime_1, op_label_bool, n_conn)
        n_conn_1 = sections[0]
        prob_corr = n_conn/n_conn_1

        
#         print(r_prime)

        prob = np.exp(
            2 * (log_values_1 - log_values)
        ) * prob_corr

        if prob > np.random.rand(1):
            r_prime[:] = r_prime_1
            state[:] = state_1
            acting_on_prime[:] = acting_on_prime_1
            conn_op_prime[:] = conn_op_prime_1

            log_values = log_values_1
            n_conn = n_conn_1
            accepted += 1
        

        N += 1
        if accepted >= sweep_size:
            break
        
    return accepted/N

In [26]:
state = np.ones(kernel.size, dtype=np.int8)
state_1 = np.copy(state)

In [27]:
%%timeit 
new_transition_2(state, state_1, ma._w, None, 10**4, kernel.basis,kernel.n_conns, kernel.x_prime, kernel.acting_on, ad2o_o, ad2_bool)

230 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
state = np.ones((1,kernel.size),dtype=np.int32)

In [29]:
%%timeit
tmp = new_transition(state, state1, ma._w, None, 10**4, kernel.basis,kernel.n_conns, kernel.x_prime, kernel.acting_on, ad2o_o.astype(np.int64), ad2_bool)

735 ms ± 6.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [147]:
tmp

0.8368901163277261

In [47]:
w = ma._w
%timeit state.dot(w)

21.5 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [614]:
op._acting_on.shape

(864, 4)

This sugests that dot calculation dominates _log_val_kernel

In [611]:
ma._w

(288, 576)